In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, NoAlertPresentException
import pandas as pd
import time
from bs4 import BeautifulSoup

# Specify your Naver Cafe URL, username, and password
cafe_url = 'https://cafe.naver.com/laterlife'
username = 'lucas4569'
password = 'Sogang0217@'

# Set up Chrome driver
driver = webdriver.Chrome()

def naver_login(driver, username, password):
    driver.get('https://nid.naver.com/nidlogin.login')
    driver.execute_script("document.getElementsByName('id')[0].value='" + username + "'")
    driver.execute_script("document.getElementsByName('pw')[0].value='" + password + "'")
    driver.find_element(by=By.XPATH, value='//*[@id="log.login"]').click()
    time.sleep(1)

def extract_content(url):
    내용 = []

    try:
        # Use WebDriverWait for the page content to load
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.NAME, "cafe_main")))

        # Wait for the content elements to be present
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.se-component.se-text.se-l-default')))

        # Find all elements with the specified class
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # Use find_all to get all elements with the specified class
        contents_elements = soup.find_all('div', class_='se-component se-text se-l-default')

        # Iterate through elements to get text content
        for element in contents_elements:
            try:
                # Extract text content and append to the list
                text_content = ' '.join(part.get_text(strip=True) for part in element.find_all(['p', 'span']))
                내용.append(text_content)
            except AttributeError:
                print(f"No text data found for URL: {url}. Skipping to the next page.")
                break

    except NoAlertPresentException:
        print(f"No alert present for URL: {url}. Skipping to the next page.")
    except TimeoutException:
        print(f"Timeout occurred while processing URL: {url}. Skipping to the next page.")
    except Exception as e:
        print(f"An error occurred while processing URL: {url}. Error: {e}")
    finally:
        driver.switch_to.default_content()

    return 내용

def save_to_excel(content_list):
    df = pd.DataFrame({"내용": content_list})
    df.to_excel("test3.xlsx", index=False)

# Log in to Naver
naver_login(driver, username, password)

# Extract URLs from the cafe
주소 = []

for i in range(401, 500 + 1):  # Adjusted the range for the next set of pages
    url_string = f'{cafe_url}?iframe_url=/ArticleList.nhn%3Fsearch.clubid=23676262%26search.boardtype=L%26search.totalCount=151%26search.cafeId=20365296%26search.page={i}'

    driver.get(url_string)
    time.sleep(0.2)
    driver.switch_to.frame("cafe_main")
    
    for j in range(1, 16, 1):
        xpath = f"//div[@class='article-board m-tcol-c']/table/tbody/tr[{j}]/td[1]/div[2]/div/a"
        try:
            url_element = driver.find_element(By.XPATH, xpath)
            url = url_element.get_attribute("href")
            주소.append(url)
        except NoSuchElementException:
            print(f"Element not found for XPath: {xpath}. Skipping to the next row.")

    # Navigate to the next page by clicking on the "다음" (Next) button
    try:
        next_page_button = driver.find_element(By.CLASS_NAME, 'pgR')
        next_page_button.click()
        time.sleep(0.2)  # Add a short delay to allow the page to load
    except NoSuchElementException:
        print("Next page button not found. Exiting loop.")

# Perform actions on the extracted URLs and save to Excel
내용 = []

for url in 주소:
    driver.get(url)
    내용.extend(extract_content(url))

# Save to Excel
save_to_excel(내용)

# Close the browser window
driver.quit()
